## Example of RBC `external` function

The `external` feature provides a way of calling external functions from UDF/UDTFs.

As a starting point of this notebook, let's connect to the OmniSciDB server and insert some test data

In [8]:
import numpy as np

In [9]:
from rbc.omniscidb import RemoteOmnisci

In [10]:
omnisci = RemoteOmnisci(user='admin', password='HyperInteractive',
                        host='127.0.0.1', port=6274, dbname='omnisci')

### Now, let's insert some test data...

Which consists of 5 negative integers 

In [11]:
SQL=lambda query: np.array(list(omnisci.sql_execute(query)[1]))
SQL('DROP TABLE IF EXISTS test_data');
SQL('CREATE TABLE IF NOT EXISTS test_data (X BIGINT)')
omnisci.load_table_columnar('test_data', X=[-1, -2, -3, -4, -5])

In [12]:
SQL('SELECT * FROM test_data')

array([[-1],
       [-2],
       [-3],
       [-4],
       [-5]])

## Declare and use external functions

RBC defines a helper function `external` which can be used to define and call external functions. e.g. function defined by the server, an external library or from the C standard library. It expects the function declaration in the following forms:

#### As a C function declaration:

```python
from rbc.external import external
fn = external('int64 abs(int64)')
```

#### Function name using a keyword argument

```python
from rbc.external import external
fn = external('int64(int64)', name='abs')
```

#### A list of signatures

```python
from rbc.external import external
fn = external(['i32(i32)', 'i64(i64)'], name='abs')
```

## Usage

One can use external function from any jitted functions. For instance, let's define and use the `abs` function from the `cmath` library:

In [28]:
from rbc.external import external
abs = external('int64 abs(int64)')

Let's now declare a function that uses the `abs`

In [29]:
@omnisci('int64(int64)')
def use_abs(i):
    return abs(i)

In [30]:
result = SQL('SELECT X, use_abs(X) FROM test_data')

In [31]:
for r, v in result:
    print(f'abs({r}) = {v}')

abs(-1) = 1
abs(-2) = 2
abs(-3) = 3
abs(-4) = 4
abs(-5) = 5


## Caveats

Notice that `abs` is not callable from pure python

In [32]:
# NBVAL_RAISES_EXCEPTION
abs(-3)

NotImplementedError: abs is not usable in pure-python